In [1]:
# pip install langgraph

from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages

# ---- Fake LLM classifier ----
def fake_classify(state):
    msg = state["messages"][-1]["content"].lower()
    if "weather" in msg:
        state["task"] = "weather"
    elif any(char.isdigit() for char in msg):
        state["task"] = "math"
    else:
        state["task"] = "unknown"
    return state

# ---- Fake Tools ----
def get_weather(location: str) -> str:
    return f"The weather in {location} is always sunny ☀️."

def do_math(expression: str) -> str:
    try:
        return str(eval(expression))
    except:
        return "Invalid math expression."

# ---- Tool handlers for LangGraph ----
def handle_weather(state):
    msg = state["messages"][-1]["content"]
    result = get_weather(msg)
    return add_messages(state, [{"role": "assistant", "content": result}])

def handle_math(state):
    msg = state["messages"][-1]["content"]
    result = do_math(msg)
    return add_messages(state, [{"role": "assistant", "content": result}])

def handle_unknown(state):
    return add_messages(state, [{
        "role": "assistant",
        "content": "❓ Sorry, I didn't understand. Can you rephrase?"
    }])

# ---- Build the graph ----
class MyState(dict):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.setdefault("messages", [])


graph = StateGraph(MyState)

# Register all nodes
graph.add_node("classify", fake_classify)
graph.add_node("weather_node", handle_weather)
graph.add_node("math_node", handle_math)
graph.add_node("unknown_node", handle_unknown)

# ✅ Set entry point
graph.set_entry_point("classify")

# Conditional routing logic
graph.add_conditional_edges(
    "classify",
    lambda state: state["task"],
    {"weather": "weather_node", "math": "math_node", "unknown": "unknown_node"}
)

# Define end nodes
graph.add_edge("weather_node", END)
graph.add_edge("math_node", END)
graph.add_edge("unknown_node", END)

# Compile the graph
app = graph.compile()

app.get_graph().print_ascii()



# ---- Run chat loop ----
def chat():
    while True:
        user_input = input("\n🧑 You: ")
        if user_input.lower() in {"exit", "quit"}:
            print("👋 Goodbye!")
            break

        state = {"messages": [{"role": "user", "content": user_input}]}
        result = app.invoke(state)
        response = result["messages"][-1]["content"]
        print("🤖 Bot:", response)

if __name__ == "__main__":
    chat()


                          +-----------+                              
                          | __start__ |                              
                          +-----------+                              
                                 *                                   
                                 *                                   
                                 *                                   
                           +----------+                              
                          .| classify |.                             
                      .... +----------+ ....                         
                  ....           .          .....                    
              ....              .                ....                
           ...                  .                    ...             
+-----------+           +--------------+           +--------------+  
| math_node |*          | unknown_node |           | weather_node |  
+-----------+ ****  


🧑 You:  what is 2 plus 2


KeyError: 'messages'

In [3]:
# pip install langgraph

from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages

# Define a custom state with default "messages" key
class MyState(dict):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.setdefault("messages", [])

# Fake LLM classifier - safely access messages
def fake_classify(state):
    messages = state.get("messages", [])
    msg = messages[-1]["content"].lower() if messages else ""

    if "weather" in msg:
        state["task"] = "weather"
    elif any(char.isdigit() for char in msg):
        state["task"] = "math"
    else:
        state["task"] = "unknown"
    return state

# Fake tool for weather
def get_weather(location: str) -> str:
    return f"The weather in {location} is always sunny ☀️."

# Fake tool for math evaluation
def do_math(expression: str) -> str:
    try:
        return str(eval(expression))
    except:
        return "Invalid math expression."

# Handle weather node
def handle_weather(state):
    messages = state.get("messages", [])
    msg = messages[-1]["content"] if messages else ""
    result = get_weather(msg)
    return add_messages(state, [{"role": "assistant", "content": result}])

# Handle math node
def handle_math(state):
    messages = state.get("messages", [])
    msg = messages[-1]["content"] if messages else ""
    result = do_math(msg)
    return add_messages(state, [{"role": "assistant", "content": result}])

# Handle unknown node
def handle_unknown(state):
    return add_messages(state, [{
        "role": "assistant",
        "content": "❓ Sorry, I didn't understand that. Can you rephrase?"
    }])

# Build the graph
graph = StateGraph(MyState)

# Register nodes
graph.add_node("classify", fake_classify)
graph.add_node("weather_node", handle_weather)
graph.add_node("math_node", handle_math)
graph.add_node("unknown_node", handle_unknown)

# Set entry point node
graph.set_entry_point("classify")

# Add conditional routing edges (task to nodes)
graph.add_conditional_edges(
    "classify",
    lambda state: state["task"],
    {"weather": "weather_node", "math": "math_node", "unknown": "unknown_node"}
)

# End nodes
graph.add_edge("weather_node", END)
graph.add_edge("math_node", END)
graph.add_edge("unknown_node", END)

# Compile the graph app
app = graph.compile()

# Chat loop
def chat():
    print("Welcome! Type 'exit' or 'quit' to stop.")
    while True:
        user_input = input("\n🧑 You: ")
        if user_input.lower() in {"exit", "quit"}:
            print("👋 Goodbye!")
            break
        
        state = {
            "messages": [{"role": "user", "content": user_input}]
        }

        try:
            result = app.invoke(state)
            response = result["messages"][-1]["content"]
        except Exception as e:
            response = f"⚠️ Error: {e}"
        print("🤖 Bot:", response)

if __name__ == "__main__":
    chat()


Welcome! Type 'exit' or 'quit' to stop.



🧑 You:  weather in india


🤖 Bot: ⚠️ Error: Message dict must contain 'role' and 'content' keys, got {}
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/MESSAGE_COERCION_FAILURE 



🧑 You:  What's the weather in Delhi?


🤖 Bot: ⚠️ Error: Message dict must contain 'role' and 'content' keys, got {}
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/MESSAGE_COERCION_FAILURE 


KeyboardInterrupt: Interrupted by user